# 端到端建模流程
## 从原始数据输入到结果文件生成

# 0.引入三方库

In [1]:
import gc
import numpy as np
import pandas as pd

import lightgbm as lgb

# 1.读取所需数据

In [2]:
debug = False
NROWS = 10000 if debug else None
train = pd.read_csv('data/train.csv', nrows = NROWS)
test =  pd.read_csv('data/test.csv', nrows = NROWS)

merchant = pd.read_csv('data/merchants.csv', nrows = NROWS)
new_transaction = pd.read_csv('data/new_merchant_transactions.csv', nrows = NROWS)
history_transaction = pd.read_csv('data/historical_transactions.csv', nrows = NROWS)

# 2.做数据预处理

In [3]:
# 商户信息
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)), index=value)).values

In [4]:
category_cols = ['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'category_4', 'city_id', 'state_id', 'category_2']
numeric_cols = ['numerical_1', 'numerical_2',
     'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12']

# 将离散字段进行编码
for col in ['category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']:
    merchant[col] = change_object_cols(merchant[col])
    
# 同时为了能够更方便统计，首先做缺失值处理，对离散字段可统一用-1进行填充
merchant[category_cols] = merchant[category_cols].fillna(-1)

# 对离散型字段探查发现有正无穷值，这也是模型所不能接受的，因此需要对无限值进行处理
inf_cols = ['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']
merchant[inf_cols] = merchant[inf_cols].replace(np.inf, merchant[inf_cols].replace(np.inf, -99).max().max())

# 对于离散字段的缺失值处理方式也有多样，这里先使用平均值进行填充，后续有需要再进行优化处理
for col in numeric_cols:
    merchant[col] = merchant[col].fillna(merchant[col].mean())
    
# 去除与transaction重复的列(当然, id显然不包括在内)
duplicate_cols = ['merchant_id', 'merchant_category_id', 'subsector_id', 'category_1', 'city_id', 'state_id', 'category_2']

In [5]:
merchant = merchant.drop(duplicate_cols[1:], axis=1)

merchant = merchant.loc[merchant['merchant_id'].drop_duplicates().index.tolist()].reset_index(drop=True)

In [6]:
# 交易记录
transaction = pd.concat([new_transaction, history_transaction], axis=0, ignore_index=True)
del new_transaction
del history_transaction
gc.collect()

numeric_cols = [ 'installments', 'month_lag', 'purchase_amount']
category_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1',
       'category_3', 'merchant_category_id', 'merchant_id', 'category_2', 'state_id',
       'subsector_id']
time_cols = ['purchase_date']

In [7]:
# 可仿照merchant的处理方式对字符型的离散特征进行编码（id除外）
for col in ['authorized_flag', 'category_1', 'category_3']:
    transaction[col] = change_object_cols(transaction[col].fillna(-1).astype(str))
transaction[category_cols] = transaction[category_cols].fillna(-1)
transaction['category_2'] = transaction['category_2'].astype(int)
# 进行时间段的处理，简单起见进行月份、日期的星期数（工作日与周末）、以及
# 时间段（上午、下午、晚上、凌晨）进行提取

In [8]:
from datetime import datetime
transaction['purchase_month'] = transaction['purchase_date'].apply(lambda x:'-'.join(x.split(' ')[0].split('-')[:2]))
transaction['purchase_hour_section'] = transaction['purchase_date'].apply(lambda x: x.split(' ')[1].split(':')[0]).astype(int)//6
transaction['purchase_day'] = transaction['purchase_date'].apply(lambda x: datetime.strptime(x.split(" ")[0], "%Y-%m-%d").weekday())//5                                                                    

In [9]:
del transaction['purchase_date']

In [10]:
# 减少内存起见，统一对merchant和transaction的id列进行编码
merchant_id = set(merchant['merchant_id'].unique().tolist() + transaction['merchant_id'].unique().tolist())
merchant_map = pd.Series(range(len(merchant_id)), index=merchant_id)

transaction['merchant_id'] = transaction['merchant_id'].map(merchant_map)
merchant['merchant_id'] = merchant['merchant_id'].map(merchant_map)

In [11]:
# 减少内存起见，统一对merchant和transaction的id列进行编码

se_map = change_object_cols(train['first_active_month'].append(test['first_active_month']).astype(str))

train['first_active_month'] = se_map[:train.shape[0]]
test['first_active_month'] = se_map[train.shape[0]:]



In [12]:
## 对购买月份进行编码
transaction['purchase_month'] = change_object_cols(transaction['purchase_month'])

In [13]:
transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'category_2', 'state_id', 'subsector_id',
       'purchase_month', 'purchase_hour_section', 'purchase_day'],
      dtype='object')

In [14]:
## 初步策略，第一步采用字典结构统计用户的所有行为特征
## 第二步，用户层面横向进行比较和特征提取
## 第三步，纵向对比所有用户的交易记录
cols = ['merchant_id', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
transaction = pd.merge(transaction, merchant[cols], how='left', on='merchant_id')
numeric_cols = ['purchase_amount', 'installments']
category_cols = ['authorized_flag', 'city_id', 'category_1',
       'category_3', 'merchant_category_id','month_lag','most_recent_sales_range',
                 'most_recent_purchases_range', 'category_4',
                 'purchase_month', 'purchase_hour_section', 'purchase_day']
id_cols = ['card_id', 'merchant_id']

In [15]:
transaction[cols[1:]] = transaction[cols[1:]]

In [16]:
transaction[category_cols].nunique()

authorized_flag                  2
city_id                        308
category_1                       2
category_3                       4
merchant_category_id           331
month_lag                       16
most_recent_sales_range          5
most_recent_purchases_range      5
category_4                       2
purchase_month                  16
purchase_hour_section            4
purchase_day                     2
dtype: int64

In [17]:
transaction[category_cols] =transaction[category_cols]

# 3.进行特征提取

In [18]:
transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'category_2', 'state_id', 'subsector_id',
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4'],
      dtype='object')

In [19]:
numeric_cols = ['authorized_flag',  'category_1', 'installments',
       'category_3',  'month_lag',
       'purchase_amount', 'category_2', 
       'purchase_month', 'purchase_hour_section', 'purchase_day',
       'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']
categorical_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id', ]
aggs = {}
for col in numeric_cols:
    aggs[col] = ['nunique', 'mean', 'min', 'max','var','skew', 'sum']
for col in categorical_cols:
    aggs[col] = ['nunique']    
aggs['card_id'] = ['size', 'count']

cols = ['card_id']
for key in aggs.keys():
    cols.extend([key+'_'+stat for stat in aggs[key]])

In [20]:
df = transaction[transaction['month_lag']<0].groupby('card_id').agg(aggs).reset_index()
df.columns = cols[:1] + [co+'_hist' for co in cols[1:]]


df2 = transaction[transaction['month_lag']>=0].groupby('card_id').agg(aggs).reset_index()
df2.columns = cols[:1] + [co+'_new' for co in cols[1:]]
df = pd.merge(df, df2, how='left',on='card_id')

df2 = transaction[transaction['month_lag']>=-3].groupby('card_id').agg(aggs).reset_index()
df2.columns = cols[:1] + [co+'_-3' for co in cols[1:]]
df = pd.merge(df, df2, how='left',on='card_id')

df2 = transaction[transaction['month_lag']>=-6].groupby('card_id').agg(aggs).reset_index()
df2.columns = cols[:1] + [co+'_-6' for co in cols[1:]]
df = pd.merge(df, df2, how='left',on='card_id')

df2 = transaction.groupby('card_id').agg(aggs).reset_index()
df2.columns = cols
df = pd.merge(df, df2, how='left',on='card_id')

In [21]:
del transaction
gc.collect()

743

# 4.进行模型训练及预测

In [22]:
train = pd.merge(train, df, how='left', on='card_id')
test =  pd.merge(test, df, how='left', on='card_id')
del df
train.to_csv("preprocess/train_groupby.csv", index=False)
test.to_csv("preprocess/test_groupby.csv", index=False)


In [23]:
train = pd.read_csv("preprocess/train_groupby.csv")
test = pd.read_csv("preprocess/test_groupby.csv")
label = 'target'
features = train.columns.tolist()
features.remove('card_id')
features.remove('target')

train = train.sample(frac=1, random_state=2020)
valid = train[int(0.8*train.shape[0]):].reset_index(drop=True)
train = train[:int(0.8*train.shape[0])].reset_index(drop=True)

# 配置模型的训练参数
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.1, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'rmse',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'regression'
    }
ESR = 30
NBR = 10000
VBE = 50

# 转换训练集与验证集为指定的数据格式
train = lgb.Dataset(train[features], train[label])
valid = lgb.Dataset(valid[features], valid[label])

# 模型训练
bst = lgb.train(params_initial,train, 
      num_boost_round=NBR, valid_sets=[train,valid],
      valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=VBE)

# 预测结果
prediction = bst.predict(test[features])
# 获取模型的特征重要性
fse = pd.Series(bst.feature_importance(), index=features)


test['target'] = prediction

submission = test[['card_id', 'target']]

Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.46222	valid's rmse: 3.59851
Early stopping, best iteration is:
[60]	train's rmse: 3.43224	valid's rmse: 3.59692


In [27]:
submission.to_csv('result/baseline_groupby_submission.csv', index=False)

In [ ]:
fse.to_csv('result/baseline_groupby_feature_importance.csv', index=False)

In [ ]:
pd.read_csv("preprocess/train_groupby.csv").head()

In [28]:
submission.head()

card_id    target
0  C_ID_0ab67a22ab -4.473419
1  C_ID_130fd0cbdd -0.585369
2  C_ID_b709037bc5 -0.202333
3  C_ID_d27d835a9f -0.015806
4  C_ID_2b5e3df5c2 -0.537914

In [ ]:
# 后续优化方向
# 1. 只用最近的交易记录(效果较差)
# 2. 特征选择(LGB用特征重要性cv，RF用相关系数等进行筛选，XGB用Stacking进行降维)
# 3. 五折
# 4. 调参（LGB用超参调优，RF用GridSearch）
# 5. 模型融合(LGB+RF+XGB,类别特征少，因此不用CATBOOST)

In [ ]:
fse.shape

In [ ]:
import pandas as pd
data =pd.read_csv("result/LGB_ParamSearch_groupby_submission.csv")

data2 =pd.read_csv("result/XGB_ParamSearch_submission.csv")

data['target2'] = data2['target'].values
print(data.corr())

data['target'] = data['target']*0.6+data['target2']*0.4
data[['card_id','target']].to_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6.csv", index=False)

In [ ]:
train = pd.read_csv("preprocess/train_groupby.csv")
test = pd.read_csv("preprocess/test_groupby.csv")
label = 'target'

In [ ]:
train[label] = (train[label]<=-33).astype(int)
train[label].value_counts()

In [ ]:
train[label].min()

In [ ]:
features = train.columns.tolist()
features.remove('card_id')
features.remove('target')

train = train.sample(frac=1, random_state=2020)
valid = train[int(0.8*train.shape[0]):].reset_index(drop=True)
train = train[:int(0.8*train.shape[0])].reset_index(drop=True)

# 配置模型的训练参数
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.1, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'auc',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'binary'
    }
ESR = 30
NBR = 10000
VBE = 50

# 转换训练集与验证集为指定的数据格式
train = lgb.Dataset(train[features], train[label])
valid = lgb.Dataset(valid[features], valid[label])

# 模型训练
bst = lgb.train(params_initial,train, 
      num_boost_round=NBR, valid_sets=[train,valid],
      valid_names=['train','valid'],early_stopping_rounds=ESR,verbose_eval=VBE)

# 预测结果
prediction = bst.predict(test[features])
# 获取模型的特征重要性
fse = pd.Series(bst.feature_importance(), index=features)


test['target'] = prediction

submission = test[['card_id', 'target']]

In [ ]:
pd.Series(prediction).describe()

In [ ]:
import pandas as pd
sub1 = pd.read_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6.csv")
sub2 = pd.read_csv("result/LGB_ParamSearch_groupby_Class_submission.csv")
sub = sub1.copy()
for k in range(30, 100, 10):
    card = sub2.sort_values("target", ascending=False)[:k]['card_id'].values.tolist()
    sub['target2'] = sub['card_id'].map(pd.Series(-33.21928095, index=card)).fillna(0)
    sub['target'] = sub['target']*((sub['target2']==0).astype(int)) + sub['target2']
    print(sub['target'].value_counts().head(1))
    sub[sub1.columns].to_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6_top"+str(k)+".csv", index=False)

In [ ]:
sub2.sort_values("target", ascending=False)

In [ ]:
import pandas as pd
data =pd.read_csv("result/LGB_ParamSearch_groupby_submission.csv")

data2 =pd.read_csv("result/LGB_ParamSearch_groupby_Drop_submission.csv")

data['target2'] = data2['target'].values
print(data.corr())

data['target'] = data['target']*0.8+data['target2']*0.2
data[['card_id','target']].to_csv("result/groupby8_groupbyDrop2.csv", index=False)

In [ ]:
import pandas as pd
sub1 = pd.read_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6.csv")
sub2 = pd.read_csv("result/LGB_ParamSearch_groupby_Class_submission.csv")
sub = sub1.copy()
for k in range(30, 100, 10):
    card = sub2.sort_values("target", ascending=False)[:k]
    card['target'] = card['target']*(-33.21928095)
    sub['target2'] = sub['card_id'].map(pd.Series(card['target'].values, index=card['card_id'].values)).fillna(0)
    sub['target'] = sub['target']*((sub['target2']==0).astype(int)) + sub['target2']
    print(sub['target'].value_counts().head(10))
    sub[sub1.columns].to_csv("result/XGB_ParamSearch4_LGB_ParamSearch_groupby6_top"+str(k)+".csv", index=False)

In [ ]:
card = sub2.sort_values("target", ascending=False)[:k]
card['target'] = card['target']*(-33.21928095)

In [ ]:
import pandas as pd
sub1 = pd.read_csv("result/XGB_ParamSearch8_LGB_ParamSearch_groupby2.csv")
sub2 = pd.read_csv("result/LGB_ParamSearch_groupby_Class_submission.csv")
sub = sub1.copy()
sub['target'] = sub2['target']*(-33.21928095) + (1-sub2['target'])*sub1['target']
sub[sub1.columns].to_csv("result/XGB_ParamSearch2_LGB_ParamSearch_groupby8_Class_Trick.csv", index=False)

In [ ]:
import pandas as pd
sub1 = pd.read_csv("result/LGB_ParamSearch_groupby_Drop_submission.csv")
sub2 = pd.read_csv("result/LGB_ParamSearch_groupby_Class_submission.csv")
sub = sub1.copy()
sub['target'] = sub2['target']*(-33.21928095) + (1-sub2['target'])*sub1['target']
sub[sub1.columns].to_csv("result/Class_Drop_Trick.csv", index=False)

In [ ]:
sub['target'].describe()

In [ ]:
import pandas as pd
data =pd.read_csv("result/Class_Drop_Trick.csv")

data2 =pd.read_csv("result/XGB_ParamSearch8_LGB_ParamSearch_groupby2.csv")

data['target2'] = data2['target'].values
print(data.corr())

data['target'] = data['target']*0.5+data['target2']*0.5
data[['card_id','target']].to_csv("result/ronghe.csv", index=False)